# Import library

In [1]:
import cv2
import os
import random
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from keras.utils import normalize
from PIL import Image
from sklearn.model_selection import train_test_split

### Avoid difference result after rerun notebook

In [2]:
seed = 99
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Reading dataset

In [3]:
no_dir = os.listdir('./data_no/data_no/NO/')
yes_dir = os.listdir('./data_yes/data_yes/YES/')

In [4]:
len(no_dir)

535

In [5]:
len(yes_dir)

557

In [6]:
data_set,label = [],[]
for i,cur_img_dir in enumerate(no_dir):
    #check type of image
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./data_no/data_no/NO/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        data_set.append(np.array(img))
        label.append(0)

In [7]:
for i,cur_img_dir in enumerate(yes_dir):
    #check type of image
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./data_yes/data_yes/YES/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        data_set.append(np.array(img))
        label.append(1)

In [8]:
data_set = np.array(data_set)
label = np.array(label)
data_set.shape

(1092, 64, 64, 3)

In [9]:
label.shape

(1092,)

In [10]:
print(f'yes observe:{sum(label)}, no observe:{len(label)-sum(label)}')

yes observe:557, no observe:535


# Split and normalize data


## 1. Split


In [11]:
x_train,x_test,y_train,y_test = train_test_split(
    data_set,label,
    test_size=0.2,
    random_state=99
    )
x_train,x_val,y_train,y_val = train_test_split(
        x_train,y_train,
    test_size=0.25,
    random_state=99
)


In [12]:
print(f'X train shape: {x_train.shape}\nY train shape: {y_train.shape}\nX test shape: {x_test.shape}\nY test shape: {y_test.shape}\nX validation shape: {x_val.shape}\nY validation shape: {x_val.shape}')

X train shape: (654, 64, 64, 3)
Y train shape: (654,)
X test shape: (219, 64, 64, 3)
Y test shape: (219,)
X validation shape: (219, 64, 64, 3)
Y validation shape: (219, 64, 64, 3)


## 2. Normalize

In [13]:
x_train = normalize(x_train, axis=1)
x_test = normalize(x_test, axis=1)
x_val = normalize(x_val, axis=1)

# Building Model

In [14]:
from keras.models import Sequential 
from tensorflow.keras.optimizers import Adam
from keras.layers import (
    Conv2D,
    MaxPooling2D,
    Activation,
    Dropout,
    Flatten,
    Dense
)

In [17]:
model=Sequential()
INPUT_SIZE = 64

model.add(Conv2D(32,(3,3), input_shape=(INPUT_SIZE,INPUT_SIZE,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Training model

### F1 score function

In [18]:
# Define custom F1 score metric
def f1_score(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    return f1_val

In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(amsgrad=True), 
    metrics=[
        'accuracy',
        tf.keras.metrics.TruePositives(name='true_positives'),
        tf.keras.metrics.TrueNegatives(name='true_negatives'),
        tf.keras.metrics.FalseNegatives(name='false_negatives'),
        tf.keras.metrics.FalsePositives(name='false_positives'),
        f1_score
    ]
)

# Model checkpoints
filepath = "weights-improvement-{epoch:02d}-{val_f1_score:.2f}.weights.h5"
savemodel = '/model' 
checkpoint = ModelCheckpoint(
    os.path.join(savemodel, filepath),
    monitor='val_f1_score',
    verbose=1,
    save_best_only=False,
    save_weights_only=True,
    mode='auto'
)

# Ensure y_train and y_val are of type float32
y_train = y_train.astype(np.float32)
y_val = y_val.astype(np.float32)

# Fit model
callbacks_list = [checkpoint]
history = model.fit(
    x_train,
    y_train,
    batch_size=16,
    verbose=1,
    epochs=10,
    validation_data=(x_val, y_val),
    shuffle=False,
    callbacks=callbacks_list
)

Epoch 1/10
40/41 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5359 - f1_score: 8.2010 - false_negatives: 52.0250 - false_positives: 93.4000 - loss: 0.6955 - true_negatives: 63.7750 - true_positives: 118.8000
Epoch 1: saving model to /model\weights-improvement-01-9.47.weights.h5
41/41 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.5380 - f1_score: 8.2148 - false_negatives: 54.0714 - false_positives: 97.5238 - loss: 0.6947 - true_negatives: 67.1190 - true_positives: 124.8095 - val_accuracy: 0.6164 - val_f1_score: 9.4681 - val_false_negatives: 1.0000 - val_false_positives: 83.0000 - val_loss: 0.6315 - val_true_negatives: 32.0000 - val_true_positives: 103.0000
Epoch 2/10
40/41 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7060 - f1_score: 8.5159 - false_negatives: 42.6000 - false_positives: 51.3250 - loss: 0.5771 - true_negatives: 105.8500 - true_positives: 128.2250
Epoch 2: saving model to /model\weights-improvement-02-7.70.weights.h5
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accura

# Test with new data

In [20]:
y_test_pred = model.predict(x_test)
print(y_test_pred.shape)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
(219, 1)


In [21]:
y_pred = (y_test_pred >0.5).astype(int)
y_pred.shape

(219, 1)

In [22]:
y_test.shape

(219,)

In [23]:
y_test_reshape = y_test.reshape(-1,1)
y_test_reshape.shape

(219, 1)

In [24]:
print("Accuracy in test set:", accuracy_score(y_test_reshape, y_pred))
print('Accuracy in validation set:',history.history['val_accuracy'][-1])

Accuracy in test set: 0.9406392694063926
Accuracy in validation set: 0.9589040875434875


# Save original model 

**Infor**<br>
Accuracy in test set: 0.9406392694063926
Accuracy in validation set: 0.9589040875434875
Name: BrainTurmor_v2<br>

Status: Good

# Save original model 

In [25]:
model.save('BrainTurmor_v1.keras')